
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

## Задача поиска схожих по смыслу предложений

Мы будем ранжировать вопросы [StackOverflow](https://stackoverflow.com) на основе семантического векторного представления 

Введем математическую формулировку задачи ранжировния

## Задача ранжирования (Learning to Rank)

* $X$ - множество объектов
* $X^l = \{x_1, x_2, ..., x_l\}$ - обучающая выборка
<br>На обучающей выборке задан порядок между некоторыми элементами, то есть нам известно, что некий объект выборки более релевантный для нас, чем другой:
* $i \prec j$ - порядок пары индексов объектов на выборке $X^l$ c индексами $i$ и $j$
### Задача:
построить ранжирующую функцию $a$ : $X \rightarrow R$ такую, что
$$i \prec j \Rightarrow a(x_i) < a(x_j)$$

<img src="https://d25skit2l41vkl.cloudfront.net/wp-content/uploads/2016/12/Featured-Image.jpg" width=500, height=450>

### Embeddings

Будем использовать предобученные векторные представления слов на постах Stack Overflow.<br>
[A word2vec model trained on Stack Overflow posts](https://github.com/vefstathiou/SO_word2vec)

In [ ]:
!wget https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1

--2022-10-19 09:59:13--  https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1
Resolving zenodo.org (zenodo.org)... 188.184.117.155
Connecting to zenodo.org (zenodo.org)|188.184.117.155|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1453905423 (1.4G) [application/octet-stream]
Saving to: ‘SO_vectors_200.bin?download=1’

SO_vectors_200.bin? 100%[===================>]   1.35G  23.7MB/s    in 2m 8s   

2022-10-19 10:01:24 (10.8 MB/s) - ‘SO_vectors_200.bin?download=1’ saved [1453905423/1453905423]



In [ ]:
from gensim.models.keyedvectors import KeyedVectors
wv_embeddings = KeyedVectors.load_word2vec_format("SO_vectors_200.bin?download=1", binary=True)

#### Как пользоваться этими векторами?

Посмотрим на примере одного слова, что из себя представляет embedding

In [ ]:
word = 'dog'
if word in wv_embeddings:
    print(wv_embeddings[word].dtype, wv_embeddings[word].shape)

float32 (200,)


In [ ]:
print(f"Num of words: {len(wv_embeddings.index2word)}")

Num of words: 1787145


Найдем наиболее близкие слова к слову `dog`:

In [ ]:
most_similar = wv_embeddings.most_similar('dog', topn = 100)
i = 1
for word in most_similar:
  if word[0] == 'cat':
    print('Place:', i, ', sim:', word[1])
    break
  
  i += 1


Place: 26 , sim: 0.6852341294288635


In [ ]:
most_similar[:26]

[('animal', 0.8564180135726929),
 ('dogs', 0.7880867123603821),
 ('mammal', 0.7623804807662964),
 ('cats', 0.7621253728866577),
 ('animals', 0.760793924331665),
 ('feline', 0.7392398118972778),
 ('bird', 0.7315489053726196),
 ('animal1', 0.7219215631484985),
 ('doggy', 0.7213349938392639),
 ('labrador', 0.7209131717681885),
 ('canine', 0.7209056615829468),
 ('meow', 0.7185295820236206),
 ('cow', 0.7080444693565369),
 ('dog2', 0.7057910561561584),
 ('woof', 0.7050611972808838),
 ('dog1', 0.7038840055465698),
 ('dog3', 0.701882004737854),
 ('penguin', 0.6970292329788208),
 ('bulldog', 0.6940488815307617),
 ('mammals', 0.6931389570236206),
 ('bark', 0.6913799047470093),
 ('fruit', 0.6892251968383789),
 ('reptile', 0.6891210079193115),
 ('furry', 0.6863498687744141),
 ('carnivore', 0.6862949728965759),
 ('cat', 0.6852341294288635)]

### Векторные представления текста

Перейдем от векторных представлений отдельных слов к векторным представлениям вопросов, как к **среднему** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор.

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
import string

In [ ]:
import numpy as np
class MyTokenizer:
    def __init__(self):
        self.punctuation = string.punctuation
        self.stopwords = stopwords.words('english')
        
    def tokenize(self, text):
        tmp_tokens = word_tokenize(text.lower())
        tokens = [word for word in tmp_tokens if (word not in self.punctuation) and (word not in self.stopwords)]
        return tokens

In [ ]:
tokenizer = MyTokenizer()

In [ ]:
tokenizer.tokenize('I follow the Moskva, Down to Gorky Park, Listening to the wind of change')

['i',
 'follow',
 'the',
 'moskva',
 'down',
 'to',
 'gorky',
 'park',
 'listening',
 'to',
 'the',
 'wind',
 'of',
 'change']

In [ ]:
def question_to_vec(question, embeddings, tokenizer, dim=200):
    """
        question: строка
        embeddings: наше векторное представление
        dim: размер любого вектора в нашем представлении
        
        return: векторное представление для вопроса
    """
    result = []
    tokens = tokenizer.tokenize(question)
    for t in tokens:
      if t in embeddings:
        result.append(embeddings[t])
      else:
        continue
    
    if len(result) == 0:
      return np.array([0]*dim)
    else:
      return np.array(result).mean(axis=0)

Теперь у нас есть метод для создания векторного представления любого предложения.

### Оценка близости текстов

Представим, что мы используем идеальные векторные представления слов. Тогда косинусное расстояние между дублирующими предложениями должно быть меньше, чем между случайно взятыми предложениями. 

Сгенерируем для каждого из $N$ вопросов $R$ случайных отрицательных примеров и добавим к ним также положительный дубликат. Для каждого вопроса будем ранжировать с помощью нашей модели $R + 1$ примеров и смотреть на позицию дубликата. Мы хотим, чтобы дубликат был первым в ранжированном списке.

#### Hits@K
Первой простой метрикой будет количество корректных попаданий для какого-то $K$:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [rank\_q_i^{'} \le K],$$
* $\begin{equation*}
[x < 0 ] \equiv 
 \begin{cases}
   1, &x < 0\\
   0, &x \geq 0
 \end{cases}
\end{equation*}$ - индикаторная функция
* $q_i$ - $i$-ый вопрос
* $q_i^{'}$ - его дубликат
* $rank\_q_i^{'}$ - позиция дубликата в ранжированном списке ближайших предложений для вопроса $q_i$.

#### DCG@K
Второй метрикой будет упрощенная DCG метрика, учитывающая порядок элементов в списке путем домножения релевантности элемента на вес равный обратному логарифму номера позиции::
$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$
С такой метрикой модель штрафуется за большой ранк корректного ответа

<img src='https://hsto.org/files/1c5/edf/dee/1c5edfdeebce4b71a86bdf986d9f88f2.jpg' width=400, height=200>

#### Пример оценок

Вычислим описанные выше метрики для игрушечного примера. 
Пусть
* $N = 1$, $R = 3$
* <font color='green'>"Что такое python?"</font> - вопрос $q_1$
* <font color='red'>"Что такое язык python?"</font> - его дубликат $q_i^{'}$

Пусть модель выдала следующий ранжированный список кандидатов:

1. "Как изучить с++?"
2. <font color='red'>"Что такое язык python?"</font>
3. "Хочу учить Java"
4. "Не понимаю Tensorflow"

$\Rightarrow rank\_q_i^{'} = 2$

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [rank\_q_i^{'} \le 1)] = 0$
- [K = 4] $\text{Hits@4} =  [rank\_q_i^{'} \le 4] = 1$

Вычислим метрику *DCG@K* для *K = 1, 4*:
- [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = 0$
- [K = 4] $\text{DCG@4} = \frac{1}{\log_2(1+2)}\cdot[2 \le 4] = \frac{1}{\log_2{3}}$

### HITS\_COUNT и DCG\_SCORE

Каждая функция имеет два аргумента: $dup\_ranks$ и $k$. 

$dup\_ranks$ является списком, который содержит рейтинги дубликатов(их позиции в ранжированном списке).

Например, $dup\_ranks = [2]$ для примера, описанного выше.

In [ ]:
def hits_count(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть  Hits@k
    """
    hits_value = 0

    for dup in dup_ranks:
      if dup <= k:
        hits_value += 1

    return hits_value/len(dup_ranks)  

In [ ]:
def dcg_score(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть DCG@k
    """
    dcg_value = 0

    for dup in dup_ranks:
      if dup <= k:
        dcg_value += 1/np.log2(1+dup)


    return dcg_value/len(dup_ranks)

Протестируем функции. Пусть $N = 1$, то есть один эксперимент. Будем искать копию вопроса и оценивать метрики.

In [ ]:
import pandas as pd

In [ ]:
copy_answers = ["How does the catch keyword determine the type of exception that was thrown",]

# наги кандидаты
candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                       "How does the catch keyword determine the type of exception that was thrown",
                       "NSLog array description not memory address",
                       "PECL_HTTP not recognised php ubuntu"],]
# dup_ranks — позиции наших копий, так как эксперимент один, то этот массив длины 1
dup_ranks = [2]

# вычисляем метрику для разных k
print('Ваш ответ HIT:', [hits_count(dup_ranks, k) for k in range(1, 5)])
print('Ваш ответ DCG:', [round(dcg_score(dup_ranks, k), 5) for k in range(1, 5)])

Ваш ответ HIT: [0.0, 1.0, 1.0, 1.0]
Ваш ответ DCG: [0.0, 0.63093, 0.63093, 0.63093]


У вас должно получиться

In [ ]:
# correct_answers - метрика для разных k
correct_answers = pd.DataFrame([[0, 1, 1, 1], [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]],
                               index=['HITS', 'DCG'], columns=range(1,5))
correct_answers

,1,2,3,4
HITS,0,1.00000,1.00000,1.00000
DCG,0,0.63093,0.63093,0.63093


### Данные
[arxiv link](https://drive.google.com/file/d/1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_/edit)

`train.tsv` - выборка для обучения.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>**

`validation.tsv` - тестовая выборка.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...**

In [ ]:
!unzip stackoverflow_similar_questions.zip

Archive:  stackoverflow_similar_questions.zip
   creating: data/
  inflating: data/.DS_Store          
   creating: __MACOSX/
   creating: __MACOSX/data/
  inflating: __MACOSX/data/._.DS_Store  
  inflating: data/train.tsv          
  inflating: data/validation.tsv     


Считайте данные.

In [ ]:
def read_corpus(filename):
    data = []
    for line in open(filename, encoding='utf-8'):
        data.append(line.split('\t'))
    return data

Нам понадобиться только файл validation.<br> В каждой строке через \t записаны: **<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...**


In [ ]:
validation_data = read_corpus('./data/validation.tsv')

Кол-во строк

In [ ]:
len(validation_data)

3760

Размер нескольких первых строк

In [ ]:
for i in range(5):
    print(i + 1, len(validation_data[i]))

1 1001
2 1001
3 1001
4 1001
5 1001


### Ранжирование без обучения

Реализуйте функцию ранжирования кандидатов на основе косинусного расстояния. Функция должна по списку кандидатов вернуть отсортированный список пар (позиция в исходном списке кандидатов, кандидат). При этом позиция кандидата в полученном списке является его рейтингом (первый - лучший). Например, если исходный список кандидатов был [a, b, c], и самый похожий на исходный вопрос среди них - c, затем a, и в конце b, то функция должна вернуть список **[(2, c), (0, a), (1, b)]**.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from copy import deepcopy

In [ ]:
def rank_candidates(question, candidates, embeddings, tokenizer, dim=200):
    """
        question: строка
        candidates: массив строк(кандидатов) [a, b, c]
        result: пары (начальная позиция, кандидат) [(2, c), (0, a), (1, b)]
    """
    result = []
    vec_question = question_to_vec(question, embeddings, tokenizer, dim).reshape(1, -1)
    
    for i, cand in enumerate(candidates):
      vec_cand = question_to_vec(cand, embeddings, tokenizer, dim).reshape(1, -1)
      dist = cosine_similarity(vec_question, vec_cand)
      result.append((i, cand, dist))

    result.sort(key = lambda x: x[2], reverse=True)

    final_res = []
    for r in result:
      final_res.append(r[:2])
    return final_res

Протестируйте работу функции на примерах ниже. Пусть $N=2$, то есть два эксперимента

In [ ]:
questions = ['converting string to list', 'Sending array via Ajax fails'] 

candidates = [['Convert Google results object (pure js) to Python object', # первый эксперимент
               'C# create cookie from string and send it',
               'How to use jQuery AJAX for an outside domain?'],
              
              ['Getting all list items of an unordered list in PHP',      # второй эксперимент
               'WPF- How to update the changes in list item of a list',
               'select2 not displaying search results']]

In [ ]:
for question, q_candidates in zip(questions, candidates):
        ranks = rank_candidates(question, q_candidates, wv_embeddings, tokenizer)
        print(ranks)
        print()

[(1, 'C# create cookie from string and send it'), (0, 'Convert Google results object (pure js) to Python object'), (2, 'How to use jQuery AJAX for an outside domain?')]

[(0, 'Getting all list items of an unordered list in PHP'), (1, 'WPF- How to update the changes in list item of a list'), (2, 'select2 not displaying search results')]



Для первого экперимента вы можете полностью сравнить ваши ответы и правильные ответы. Но для второго эксперимента два ответа на кандидаты будут <b>скрыты</b>(*)

In [ ]:
# должно вывести
results = [[(1, 'C# create cookie from string and send it'),
            (0, 'Convert Google results object (pure js) to Python object'),
            (2, 'How to use jQuery AJAX for an outside domain?')]] 

Теперь мы можем оценить качество нашего метода. Запустите следующие два блока кода для получения результата. Обратите внимание, что вычисление расстояния между векторами занимает некоторое время (примерно 10 минут). Можете взять для validation 1000 примеров.

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [ ]:
for k in [1, 5, 10, 100, 500, 750, 1000]:
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

DCG@   1: 0.403 | Hits@   1: 0.403
DCG@   5: 0.488 | Hits@   5: 0.566
DCG@  10: 0.511 | Hits@  10: 0.636
DCG@ 100: 0.556 | Hits@ 100: 0.859
DCG@ 500: 0.571 | Hits@ 500: 0.971
DCG@ 750: 0.574 | Hits@ 750: 0.993
DCG@1000: 0.575 | Hits@1000: 1.000


### Эмбеддинги, обученные на корпусе похожих вопросов

В каждой строке через \t записаны: **<вопрос>, <похожий вопрос>**

In [ ]:
train_data_separated = read_corpus('./data/train.tsv')

Улучшите качество модели.<br>Склеим вопросы в пары и обучим на них модель Word2Vec из gensim. Выберите размер window. Объясните свой выбор.

In [ ]:
train_data = []
for w in tqdm(train_data_separated):
  train_data.append(w[0] + ' ' + w[1])

  0%|          | 0/1000000 [00:00<?, ?it/s]

In [ ]:
from gensim.models import Word2Vec
embeddings_trained = Word2Vec(train_data, # data for model to train on
                 size=200,                 # embedding vector size
                 min_count= 10,             # consider words that occured at least 20 times
                 window=7).wv

In [ ]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, tokenizer, dim = 200)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [ ]:
for k in [1, 5, 10, 100, 500, 1000]:
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

DCG@   1: 0.948 | Hits@   1: 0.948
DCG@   5: 0.950 | Hits@   5: 0.951
DCG@  10: 0.950 | Hits@  10: 0.952
DCG@ 100: 0.958 | Hits@ 100: 0.997
DCG@ 500: 0.958 | Hits@ 500: 0.997
DCG@1000: 0.959 | Hits@1000: 1.000


* Возможное улучшение - стемминг


In [ ]:
import nltk
from nltk.stem.snowball import SnowballStemmer
  
#the stemmer requires a language parameter
snow_stemmer = SnowballStemmer(language='english')
train_data_stemmed = []
for w in tqdm(train_data_separated):
  train_data_stemmed.append(' '.join([snow_stemmer.stem(word) for word in w[0].split()]) + ' ' +  ' '.join([snow_stemmer.stem(word) for word in w[1].split()]))

  0%|          | 0/1000000 [00:00<?, ?it/s]

In [ ]:
print('It was:\t', *train_data_separated[0], 'It became:', train_data_stemmed[0])
print('It was:', *train_data_separated[1], 'It became:', train_data_stemmed[1])
print('It was:', *train_data_separated[2], 'It became:', train_data_stemmed[2])

It was:	 converting string to list Convert Google results object (pure js) to Python object
 It became: convert string to list convert googl result object (pure js) to python object
It was: Which HTML 5 Canvas Javascript to use for making an interactive drawing tool? Event handling for geometries in Three.js?
 It became: which html 5 canva javascript to use for make an interact draw tool? event handl for geometri in three.js?
It was: Sending array via Ajax fails Getting all list items of an unordered list in PHP
 It became: send array via ajax fail get all list item of an unord list in php


In [ ]:
from gensim.models import Word2Vec
embeddings_trained = Word2Vec(train_data_stemmed, # data for model to train on
                 size=200,                 # embedding vector size
                 min_count= 10,             # consider words that occured at least 20 times
                 window=7).wv

In [ ]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, tokenizer, dim = 200)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [ ]:
for k in [1, 5, 10, 100, 500, 1000]:
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

DCG@   1: 0.948 | Hits@   1: 0.948
DCG@   5: 0.950 | Hits@   5: 0.951
DCG@  10: 0.950 | Hits@  10: 0.952
DCG@ 100: 0.958 | Hits@ 100: 0.997
DCG@ 500: 0.958 | Hits@ 500: 0.997
DCG@1000: 0.959 | Hits@1000: 1.000


## Вывод:
* Я использую nltk тонекизацию, убираю знаки пунктуации и стоп-слова. Выбор алгоритма токенизация в среднем слабо влияет на результат. 

* Нормализация (лемматизация или стемминг) слов в данном конкретном случае не помогли улучшить результат.

* Лучшее качество показывают эмбеддинги с большим параметра window. Увеличение параметр min_count приводит к незначительному росту метрик.

* Плохое качество могло быть вызвано применением word2vec (не хватило размеров словаря, игнорируется различное значение слов в зависимости от контекста, слабость на неизвестных словах). Также есть подозрение на сложность данных - большое количество выражений, для которых нет возможность правильно провести нормализацию и токенизацию.

* В качестве улучшения решения можно предложить разобраться с недостатками имеющегося - изменить модель, использовать более длинные эмбеддинги, поработать с корпусом преложений.